In [1]:
import sys, os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from general_tools import audio_to_spectrogram, extrait_aleatoire, rename_sounds, copy_paste

### Récupérer les extraits aléatoires de 15s

In [2]:
blues_path = os.path.join(os.getcwd(), "blues")
classique_path = os.path.join(os.getcwd(), "classique")
electro_path = os.path.join(os.getcwd(), "electro")
hardrock_path = os.path.join(os.getcwd(), "hardrock")
techno_path = os.path.join(os.getcwd(), "techno")
jazz_path = os.path.join(os.getcwd(), "jazz")
discofs_path = os.path.join(os.getcwd(), "discofs")

#classique_path = os.path.join(os.getcwd(), "classique")
#classique_path = os.path.join(os.getcwd(), "classique")
#classique_path = os.path.join(os.getcwd(), "classique")

In [3]:
#rename_sounds("hardrock")

In [4]:
# 5 extraits de 15 sec

#duree_extrait = 15 * 1000
#nbr_extraits = 5

#for sound in os.listdir(electro_path):

#    for n in range(nbr_extraits):

#        extrait = extrait_aleatoire(fichier_mp3=os.path.join(electro_path, sound), duree_extrait=duree_extrait)
#        extrait.export(os.path.join(os.getcwd(), "electro_extraits", f"extrait {n} {sound}"), format="mp3")

### Création train set / test set & labelisation 

In [5]:
spectro_path = os.path.join(os.getcwd(), "spectogrammes")

extraits_blues_path = os.path.join(os.getcwd(), "blues_extraits")
extraits_classique_path = os.path.join(os.getcwd(), "classique_extraits")
extraits_electro_path = os.path.join(os.getcwd(), "electro_extraits")
extraits_hardrock_path = os.path.join(os.getcwd(), "hardrock_extraits")
extraits_techno_path = os.path.join(os.getcwd(), "techno_extraits")
extraits_jazz_path = os.path.join(os.getcwd(), "jazz_extraits")
extraits_discofs_path = os.path.join(os.getcwd(), "discofs_extraits")

#extraits_classique_path = os.path.join(os.getcwd(), "classique_extraits")
#extraits_classique_path = os.path.join(os.getcwd(), "classique_extraits")
#extraits_classique_path = os.path.join(os.getcwd(), "classique_extraits")

##### Labelisation

In [6]:
df_data = pd.DataFrame(columns=["name", "label"])
df_data_tmp = df_data.copy(deep=True)

for (i, extraits_register_path) in enumerate([extraits_blues_path, extraits_classique_path, extraits_electro_path, extraits_hardrock_path, extraits_techno_path, extraits_jazz_path,
                               extraits_discofs_path]):

    df_data_tmp["name"] = os.listdir(extraits_register_path)
    df_data_tmp["label"] = np.ones(len(os.listdir(extraits_register_path))) * i

    df_data = pd.concat([df_data, df_data_tmp])

df_data["label"] = df_data["label"].astype(int)

/tmp/ipykernel_13176/3230694021.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data = pd.concat([df_data, df_data_tmp])


In [7]:
dic_label = {"blues": 0, "classique": 1, "electro": 2, "hardrock": 3, "techno": 4, "jazz": 5, "discofs": 6}

registers = list(dic_label.keys())
labels = list(dic_label.values())

##### Création des spectogrammes

In [6]:
old_size, new_size = 0, 0

for (i, extraits_register_path) in enumerate([extraits_blues_path, extraits_classique_path, extraits_electro_path, extraits_hardrock_path, extraits_techno_path, extraits_jazz_path,
                               extraits_discofs_path]):

    print(f"début spectro_path_size {extraits_register_path} = {len(os.listdir(spectro_path))}")

    for sound in os.listdir(extraits_register_path):

        audio_to_spectrogram(os.path.join(extraits_register_path, sound), os.path.join(spectro_path, sound+'.png'), duration=15)

        new_size = len(os.listdir(spectro_path))

        if new_size == old_size:
            print(sound)

        old_size = len(os.listdir(spectro_path))
    
    print(f"fin spectro_path_size {extraits_register_path} = {len(os.listdir(spectro_path))}")

début spectro_path_size /home/baba/Bureau/ProjetIA/blues_extraits = 0
fin spectro_path_size /home/baba/Bureau/ProjetIA/blues_extraits = 500
début spectro_path_size /home/baba/Bureau/ProjetIA/classique_extraits = 500
fin spectro_path_size /home/baba/Bureau/ProjetIA/classique_extraits = 1000
début spectro_path_size /home/baba/Bureau/ProjetIA/electro_extraits = 1000
fin spectro_path_size /home/baba/Bureau/ProjetIA/electro_extraits = 1500
début spectro_path_size /home/baba/Bureau/ProjetIA/hardrock_extraits = 1500
fin spectro_path_size /home/baba/Bureau/ProjetIA/hardrock_extraits = 2000
début spectro_path_size /home/baba/Bureau/ProjetIA/techno_extraits = 2000
fin spectro_path_size /home/baba/Bureau/ProjetIA/techno_extraits = 2500
début spectro_path_size /home/baba/Bureau/ProjetIA/jazz_extraits = 2500
fin spectro_path_size /home/baba/Bureau/ProjetIA/jazz_extraits = 3000
début spectro_path_size /home/baba/Bureau/ProjetIA/discofs_extraits = 3000
fin spectro_path_size /home/baba/Bureau/ProjetIA

In [8]:
df_data["name"] += ".png"

##### Split des data / création .csv

In [9]:
train_size = int(0.8*500)
test_size = int(0.2*500)

train_size, test_size

(400, 100)

In [10]:
X_train = pd.Series(dtype=object)

sizetmp_xtrain = 0

oui = None

for (k, num_label) in enumerate(labels):
    df_data_labels = df_data.loc[df_data["label"] == num_label].sort_values(by="name")

    for (i, sound) in enumerate(df_data_labels["name"]):
        if sound[:10].__contains__("extrait 0"):
            X_train = pd.concat([X_train, df_data[df_data["name"].apply(lambda x: sound[10:] in str(x))]["name"]])
            X_train = pd.Series(X_train.unique())
            

            if len(X_train) % train_size == 0:
                print(f"Il y a {len(X_train) - sizetmp_xtrain} audios de label {num_label}")
                sizetmp_xtrain=len(X_train)
                break

X_test = np.setdiff1d(df_data["name"].values, X_train)
X_test = pd.Series(X_test)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

Il y a 400 audios de label 0
Il y a 400 audios de label 1
Il y a 400 audios de label 2
Il y a 400 audios de label 3
Il y a 400 audios de label 4
Il y a 400 audios de label 5
Il y a 400 audios de label 6


In [27]:
path_to_train = os.path.join(os.getcwd(), "data_train")
path_to_test = os.path.join(os.getcwd(), "data_test")

for sound_train in X_train:
    copy_paste(source_path=os.path.join(spectro_path, sound_train), destination_path=os.path.join(path_to_train, sound_train))

for sound_test in X_test:
    copy_paste(source_path=os.path.join(spectro_path, sound_test), destination_path=os.path.join(path_to_test, sound_test))

In [12]:
df_data.loc[df_data["name"].isin(X_train)].to_csv("train.csv", index=False)